<a href="https://colab.research.google.com/github/xhxuciedu/cs273p/blob/main/pytorch-tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pylab as plt
import sys
import torch 
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms

In [ ]:
from IPython.display import HTML, display
def set_default(figsize=(8, 5), dpi=100):
    plt.style.use(['dark_background', 'bmh'])
    plt.rc('axes', facecolor='k')
    plt.rc('figure', facecolor='k')
    plt.rc('figure', figsize=figsize, dpi=dpi)

set_default()

### Check Package Versions

In [ ]:
print('__Python VERSION:', sys.version)
print('__PyTorch VERSION:', torch.__version__)
print('__CUDNN VERSION:', torch.backends.cudnn.version())
print('__Number CUDA Devices:', torch.cuda.device_count())

__Python VERSION: 3.7.10 (default, Feb 20 2021, 21:17:23) 
[GCC 7.5.0]
__PyTorch VERSION: 1.7.1+cu101
__CUDNN VERSION: 7603
__Number CUDA Devices: 1


### PyTorch
What is PyTorch?

It’s a Python based scientific computing package targeted at two sets of audiences:

* A replacement for numpy to use the power of GPUs
* a deep learning research platform that provides maximum flexibility and speed





### Tensors

Tensors are similar to numpy’s ndarrays, with the addition being that Tensors can also be used on a GPU to accelerate computing.


Construct a 5x3 matrix, uninitialized

In [ ]:
x = torch.Tensor(5, 3)
print(x)

tensor([[-2.3165e+14,  3.0851e-41,  1.3593e-43],
        [ 1.5975e-43,  1.3593e-43,  1.5274e-43],
        [ 1.6115e-43,  6.4460e-44,  1.3873e-43],
        [ 1.4994e-43,  1.5695e-43,  1.6255e-43],
        [ 0.0000e+00,  0.0000e+00,  3.4548e-09]])


In [ ]:
# get its size
y = torch.rand(5, 3)
print(x + y)

tensor([[-2.3165e+14,  5.9685e-01,  5.3644e-01],
        [ 9.5369e-01,  5.4011e-01,  8.1075e-01],
        [ 2.0064e-01,  7.7589e-01,  4.1524e-02],
        [ 3.7451e-01,  4.1807e-01,  1.3328e-01],
        [ 4.0639e-01,  7.4064e-01,  8.0764e-01]])


In [ ]:
# Addition: in-place
y.add_(x)

tensor([[-2.3165e+14,  5.9685e-01,  5.3644e-01],
        [ 9.5369e-01,  5.4011e-01,  8.1075e-01],
        [ 2.0064e-01,  7.7589e-01,  4.1524e-02],
        [ 3.7451e-01,  4.1807e-01,  1.3328e-01],
        [ 4.0639e-01,  7.4064e-01,  8.0764e-01]])

### Create tensors

In [ ]:
# random
v = torch.rand(2, 3)            # Initialize with random number (uniform distribution)
v = torch.randn(2, 3)           # With normal distribution (SD=1, mean=0)
v = torch.randperm(4)   

# ones
eye = torch.eye(3)              # Create an identity 3x3 tensor
v = torch.ones(10)              # A tensor of size 10 containing all ones
v = torch.ones(2, 1, 2, 1)      # Size 2x1x2x1
v = torch.ones_like(eye)        # A tensor with same shape as eye. Fill it with 1.

# zeros
v = torch.zeros(10) 

# range of values
v = torch.arange(5)             # similar to range(5) but creating a Tensor
v = torch.arange(0, 5, step=1)  # Size 5. Similar to range(0, 5, 1)

# linear or log scale
v = torch.linspace(1, 10, steps=10) # Create a Tensor with 10 linear points for (1, 10) inclusively
v = torch.logspace(start=-10, end=10, steps=5) # Size 5: 1.0e-10 1.0e-05 1.0e+00, 1.0e+05, 1.0e+10

### Dot product, component-wide product, matrix multiplication, 

In [ ]:
# Dot product of 2 tensors
r = torch.dot(torch.Tensor([4, 2]), torch.Tensor([3, 1])) # 14

In [ ]:
# component-wise product
torch.Tensor([4, 2])* torch.Tensor([3, 1])

tensor([12.,  2.])

In [ ]:
# Matrix x Matrix
# Size 2x4
mat1 = torch.randn(2, 3)
mat2 = torch.randn(3, 4)
r = torch.mm(mat1, mat2)

In [ ]:
# Batch Matrix x Matrix
# Size 10x3x5
batch1 = torch.randn(10, 3, 4)
batch2 = torch.randn(10, 4, 5)
r = torch.bmm(batch1, batch2)

###Squeeze and unsqueeze

In [ ]:
t = torch.ones(2,1,2,1) # Size 2x1x2x1
r = torch.squeeze(t)     # Size 2x2
r = torch.squeeze(t, 1)  # Squeeze dimension 1: Size 2x2x1

# Un-squeeze a dimension
x = torch.Tensor([1, 2, 3])
r = torch.unsqueeze(x, 0)       # Size: 1x3
r = torch.unsqueeze(x, 1)       # Size: 3x1

### Transpose


In [ ]:
# Transpose dim 0 and 1
v = torch.randn(3,2)
r = torch.transpose(v, 0, 1)
print(r.shape)

torch.Size([2, 3])


### Numpy Bridge
Converting a torch Tensor to a numpy array and vice versa is a breeze.

The torch Tensor and numpy array will share their underlying memory locations, and changing one will change the other.

Converting torch Tensor to numpy Array

In [ ]:
# Create a numpy array.
x = np.array([[1, 2], [3, 4]])

# Convert the numpy array to a torch tensor.
y = torch.from_numpy(x)

# Convert the torch tensor to a numpy array.
z = y.numpy()

In [ ]:
# Conversion
a = np.array([1, 2, 3])
v = torch.from_numpy(a)         # Convert a numpy array to a Tensor

b = v.numpy()                   # Tensor to numpy
b[1] = -1                       # Numpy and Tensor share the same memory
assert(a[1] == b[1])            # Change Numpy will also change the Tensor

### Reshape tensor

In [ ]:
### Tensor resizing
x = torch.randn(2, 3)            # Size 2x3
y = x.view(6)                    # Resize x to size 6
z = x.view(-1, 2)                # Size 3x2
print(y.shape, z.shape)

torch.Size([6]) torch.Size([3, 2])


###CUDA Tensors

All the Tensors on the CPU except a CharTensor support converting to NumPy and back.


Tensors can be moved onto GPU using the .cuda function.

In [ ]:
# let us run this cell only if CUDA is available

x = torch.rand(3,2)
y = torch.rand(3,2)
if torch.cuda.is_available():
    x = x.cuda()
    y = y.cuda()
    x + y

In [ ]:
x

tensor([[0.2339, 0.4187],
        [0.9977, 0.7115],
        [0.2129, 0.9028]], device='cuda:0')

In [ ]:
y

tensor([[0.1377, 0.7796],
        [0.2557, 0.0923],
        [0.3180, 0.3689]], device='cuda:0')

## Autograd: automatic differentiation

Central to all neural networks in PyTorch is autograd, a core torch package for automatic differentiation. 


The autograd package provides automatic differentiation for all operations on Tensors. It is a define-by-run framework, which means that your backprop is defined by how your code is run, and that every single iteration can be different.

Let us see this in more simple terms with some examples.

In [ ]:
# create an variable
x = torch.ones((2,2), requires_grad=True)

# Do an operation of variable:
y = x + 2

# Do more operations on y
z = y * y * 3
out = z.mean()

In [ ]:
# Gradients
# ---------
# let's backprop now
# ``out.backward()`` is equivalent to doing ``out.backward(torch.Tensor([1.0]))``
out.backward()

In [ ]:
###############################################################
# print gradients d(out)/dx
#

print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


You should have got a matrix of ``4.5``. Let’s call the ``out`` *Variable* $o$.
We have that: $o = \frac{1}{4}\sum_i z_i$, 
$z_i = 3(x_i+2)^2$ and $z_i\bigr\rvert_{x_i=1} = 27$

Therefore,
$$\frac{\partial o}{\partial x_i} = \frac{3}{2}(x_i+2)$$ hence
$$\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=1} = \frac{9}{2} = 4.5$$

In [ ]:
# You can do many crazy things with autograd!
x = torch.randn(3, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y)

tensor([-1011.5469,   212.4474,   499.2918], grad_fn=<MulBackward0>)


In [ ]:
gradients = torch.FloatTensor([0.1, 1.0, 0.0001])
y.backward(gradients)

print(x.grad)

tensor([5.1200e+01, 5.1200e+02, 5.1200e-02])


### Basic autograd example 1 

In [ ]:
# Create tensors.
x = torch.tensor(1., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)

In [ ]:
# Build a computational graph.
y = w * x + b    # y = 2 * x + 3

In [ ]:
# Compute gradients.
y.backward()

In [ ]:
# Print out the gradients.
print(x.grad)    # x.grad = 2 
print(w.grad)    # w.grad = 1 
print(b.grad)    # b.grad = 1 

tensor(2.)
tensor(1.)
tensor(1.)


In [ ]:
y.detach().numpy()

array(5., dtype=float32)

### Basic autograd example 2  

In [ ]:
# Create tensors of shape (10, 3) and (10, 2).
x = torch.randn(10, 3)
y = torch.randn(10, 2)
print(x.shape, y.shape)

torch.Size([10, 3]) torch.Size([10, 2])


In [ ]:
# Build a fully connected layer.
linear = nn.Linear(3, 2)
print ('w: ', linear.weight)
print ('b: ', linear.bias)

w:  Parameter containing:
tensor([[-0.3580, -0.1151,  0.0306],
        [-0.2708,  0.2443, -0.1761]], requires_grad=True)
b:  Parameter containing:
tensor([ 0.0616, -0.2742], requires_grad=True)


In [ ]:
loss = torch.sum((linear(x)-y)**2)/y.shape[0]
print('loss: ', loss.data.numpy())

loss:  1.0595618


In [ ]:
loss.backward()

In [ ]:
print('w grad: ', linear.weight.grad)
print('b grad: ', linear.bias.grad)

w grad:  tensor([[-1.0422,  0.0540,  0.2493],
        [-0.2615, -0.4281,  0.1976]])
b grad:  tensor([ 0.7455, -0.4274])


In [ ]:
# check grad
print('w grad:', (linear(x)-y).transpose(0,1).mm(x)/y.shape[0]*2)
print('b grad:', 2*torch.mean(linear(x)-y, dim=0))

w grad: tensor([[-1.0422,  0.0540,  0.2493],
        [-0.2615, -0.4281,  0.1976]], grad_fn=<MulBackward0>)
b grad: tensor([ 0.7455, -0.4274], grad_fn=<MulBackward0>)


###. Basic autograd example 3

In [ ]:
# Create tensors of shape (10, 3) and (10, 2).
x = torch.randn(10, 3)
y = torch.randn(10, 2)
linear = nn.Linear(3, 2)

In [ ]:
# Build loss function and optimizer.
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

# Forward pass.
pred = linear(x)

# Compute loss.
loss = criterion(pred, y)
print('loss: ', loss.item())

# Backward pass.
loss.backward()

# Print out the gradients.
print ('dL/dw: ', linear.weight.grad) 
print ('dL/db: ', linear.bias.grad)

# 1-step gradient descent.
optimizer.step()

# You can also perform gradient descent at the low level.
# linear.weight.data.sub_(0.01 * linear.weight.grad.data)
# linear.bias.data.sub_(0.01 * linear.bias.grad.data)

# Print out the loss after 1-step gradient descent.
pred = linear(x)
loss = criterion(pred, y)
print('loss after 1 step optimization: ', loss.item())

loss:  1.1785218715667725
dL/dw:  tensor([[ 0.7789,  0.2582,  0.5103],
        [-0.4900, -0.2236,  0.0021]])
dL/db:  tensor([-0.6611,  0.5150])
loss after 1 step optimization:  1.1594419479370117


## Input pipeline, Data loader

In [ ]:
# # Download and construct CIFAR-10 dataset.
# train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
#                                              train=True, 
#                                              transform=transforms.ToTensor(),
#                                              download=True)

# # Fetch one data pair (read data from disk).
# image, label = train_dataset[0]
# print (image.size())
# print (label)

# # Data loader (this provides queues and threads in a very simple way).
# train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
#                                            batch_size=64, 
#                                            shuffle=True)

# # When iteration starts, queue and thread start to load data from files.
# data_iter = iter(train_loader)

# # Mini-batch images and labels.
# images, labels = data_iter.next()

# # Actual usage of the data loader is as below.
# for images, labels in train_loader:
#     # Training code should be written here.
#     pass

Extracting ../../data/cifar-10-python.tar.gz to ../../data/
torch.Size([3, 32, 32])
6


### Input pipeline for custom dataset

In [ ]:
# # ================================================================== #
# #                  Input pipeline for custom dataset                 #
# # ================================================================== #

# # You should build your custom dataset as below.
# class CustomDataset(torch.utils.data.Dataset):
#     def __init__(self):
#         # TODO
#         # 1. Initialize file paths or a list of file names. 
#         pass
#     def __getitem__(self, index):
#         # TODO
#         # 1. Read one data from file (e.g. using numpy.fromfile, PIL.Image.open).
#         # 2. Preprocess the data (e.g. torchvision.Transform).
#         # 3. Return a data pair (e.g. image and label).
#         pass
#     def __len__(self):
#         # You should change 0 to the total size of your dataset.
#         return 0 

# # You can then use the prebuilt data loader. 
# custom_dataset = CustomDataset()
# train_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
#                                            batch_size=64, 
#                                            shuffle=True)


### Pretrained model

In [ ]:
# # ================================================================== #
# #                           Pretrained model                         #
# # ================================================================== #

# # Download and load the pretrained ResNet-18.
# resnet = torchvision.models.resnet18(pretrained=True)

# # If you want to finetune only the top layer of the model, set as below.
# for param in resnet.parameters():
#     param.requires_grad = False

# # Replace the top layer for finetuning.
# resnet.fc = nn.Linear(resnet.fc.in_features, 100)  # 100 is an example.

# # Forward pass.
# images = torch.randn(64, 3, 224, 224)
# outputs = resnet(images)
# print (outputs.size())     # (64, 100)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth



torch.Size([64, 100])


### Save and load the model    

In [ ]:
# # Save and load the entire model.
# torch.save(resnet, 'model.ckpt')
# model = torch.load('model.ckpt')

# # Save and load only the model parameters (recommended).
# torch.save(resnet.state_dict(), 'params.ckpt')
# resnet.load_state_dict(torch.load('params.ckpt'))

<All keys matched successfully>